### Author - Ajaya Kumar Sahoo

#### First compute exhaustive list of fragments using mmpdb (https://github.com/rdkit/mmpdb)
    
    - input.tsv should have the canonicalized SMILES as first column and chemical identifier as the second column
    - mmpdb fragment --max-heavies none --max-rotatable-bonds none input.tsv -o input_fragment.tsv

#### Second perform indexing of these fragment using mmpdb (https://github.com/rdkit/mmpdb)
    - mmpdb index --max-variable-heavies none input_fragment.tsv --output input_fragment_indexed.tsv

#### This code filters MMPs using different criteria proposed by Hu et al.  (https://pubs.acs.org/doi/10.1021/ci3001138)

In [1]:
import pandas as pd
from rdkit import Chem

In [3]:
df1 = pd.read_csv('input_fragment_indexed.tsv',sep='\t',dtype=str,header=None,encoding='UTF-8') # input file
print(df1.shape)
df1.columns = ['Smi1','Smi2','iden1','iden2','transformation','core']
print(df1.shape)


df1['iden1'] = df1['iden1'].str.strip()
df1['iden2'] = df1['iden2'].str.strip()

def Get_iden_trans(x):
    # this function will give the sorted combined string for identifier 
    iden1 = x['iden1']
    iden2 = x['iden2']
    return '|'.join(sorted([iden1,iden2]))

df1['iden_transform'] = df1.apply(lambda x:Get_iden_trans(x),axis=1)

print(df1.shape)
df1.head()

In [5]:
## Criteria to identify MMPs (https://pubs.acs.org/doi/10.1021/ci3001138)

# condition 1 - heavy atom counts of the components in the transformation should differ by a maximum of 8 atoms 
# condition 2 - keys (the core in the dataframe) should be at least twice the size of each component in the transformation
# condition 3 - the fragment should be <= 13
# condition 4 - for several tranaformation of a given pair, only the smallest transformation is retained


In [4]:
# condition 1

def Get_heavyatoms_num(smart):
    '''
    count the number of heavy atoms from smarts
    '''
    mol = Chem.MolFromSmarts(smart)
    num = mol.GetNumHeavyAtoms()
    return num

def Check_heavy_atom_count(x):
    '''
    compute the number of heavy atoms for transformation fragments
    Then check the absolute difference 
    return 1 if the difference <= 8 else 0
    '''
    transformation = str(x['transformation']) # take the transformation
    frag_num = list(map(Get_heavyatoms_num,transformation.split('>>'))) # apply Get_heavyatoms_num function
    return(int(abs(frag_num[0] - frag_num[1]) <= 8)) # compare the difference with the cutoff 8

df1['frag_num_diff'] = df1.apply(lambda x: Check_heavy_atom_count(x),axis=1)
print(df1.shape)
df1.head()

In [5]:
# Filter the dataframe which has frag_num_diff == 1; satisfy the condition 1

df2 = pd.DataFrame(df1[df1['frag_num_diff']==1].reset_index(drop=True))
print(df2.shape)
df2.head()

In [6]:
# condition 2

def Check_condition2(core_num,frag_num):
    '''
    checks if the number of heavy atoms in the core >= 2*(the number of heavy atoms) for both the fragments
    core_num is a number
    frag_num is a list containing numbers
    '''
    frag_num_2 = [2*i for i in frag_num] # multiply 2 to each element in the list
    frag_num_comp = [core_num >= k for k in frag_num_2] # check if the number of heavy atoms in the core >= 2*(the number of heavy atoms)
    return (int(all(frag_num_comp)))  # frag_num_comp contains True or False, all(frag_num_comp) ==1 if all are True, else 0
    
def Compare_core_transformation_fragment(x):
    '''
    This function will compare the number of heavy atoms with the number of heavy atoms in each fragment/component of the transformation
    if for both the component in the transformation the number of heavy atoms are less than that of the core, it will give a value 1, else 0
    '''
    core = str(x['core']) # reading the core
    transformation = str(x['transformation']) # reading the transformation
    heavys = list(map(Get_heavyatoms_num, transformation.split('>>'))) # apply Get_heavyatoms_num to each fragment of transformation
    core_heavy = Get_heavyatoms_num(core)
    flag = Check_condition2(core_heavy,heavys)
    return flag

df2['Compare_core_fragments'] = df2.apply(lambda x: Compare_core_transformation_fragment(x), axis=1)

print(df2.shape)
df2.head()

In [7]:
# Filter the dataframe which has Compare_core_fragments == 1; satisfy the condition 2

df3 = pd.DataFrame(df2[df2['Compare_core_fragments']==1].reset_index(drop=True))
print(df3.shape)
df3.head()

In [8]:
# condition 3 
def maximum_fragment_size(x):
    '''
    This function will check the number of heavy atoms in each fragment/component of the transformation
    if the number of heavy atoms in each fragment <= 13 it will give a value 1, else 0
    '''
    
    transformation = str(x['transformation']) # reading the transformation
    heavys = list(map(Get_heavyatoms_num, transformation.split('>>'))) # apply Get_heavyatoms_num to each fragment of transformation
    if max(heavys) > 13:
        flag = 0
    else:
        flag = 1
                
    return flag

In [9]:
# filter based on condition 3
    
df3['max_heavy_fragments'] = df3.apply(lambda x: maximum_fragment_size(x), axis=1)
print(df3.shape)
df3.head()
df3 = pd.DataFrame(df3[df3['max_heavy_fragments']==1].reset_index(drop=True))
print(df3.shape)
df3.head()

In [11]:
# Group the transformation based on the identifier

df4 = pd.DataFrame(df3[['iden_transform', 'transformation']]) # I am not considering the iden1 and iden2. I can merge them later
print(df4.shape)
print(len(set(list(df3['iden_transform']))))
df4 = df4.groupby('iden_transform').agg(lambda x: '|'.join(sorted([i for i in list(set(x)) if i != '']))).reset_index() # grouping the transformation with same id

print(df4.shape)
df4.head()

In [12]:
# condition 4

def Get_transformation_smallest_deviation(pairs):
    '''
    For a given pair, this program will consider all the possible transformation V1>>V2|V3>>V4
    Then, it will compute the number of heavy atoms for both sides of the transformation;|V1| = number of heavy atoms in V1
    for a given transformation V1>>V2, compute the absolute deviation of the heavy atom counts between the exchanged groups
    this is given by absolute deviation of the heavy atom counts = abs(|V1|-|V2|)
    Keep the exchanged group with the least number of absolute deviation of the heavy atom counts.
    '''
    groups = str(pairs['transformation']).split('|') # this will give [V1>>V2, V3>>V4]
    
    count_dic = {}
    
    for group in groups: # consider V1>>V2
        fragments = group.split('>>') # this will give [V1,V2]        
        
        atom_count = list(map(Get_heavyatoms_num,fragments))
        
        dev_heavy = abs(atom_count[0] - atom_count[1]) # this is the absolute deviation of the heavy atom count in the exchanged group
        count_dic[group] = dev_heavy # store the absolute deviation of the heavy atom counts for each exchanged groups  
    
    groups_f = []
    for k in count_dic.keys():
        if count_dic[k] == min(list(count_dic.values())): # key with the minimum absolute deviation of the heavy atom counts for exchanged groups
            groups_f.append(k)
    return '|'.join(groups_f)        

In [13]:
df4['groups'] = df4.apply(lambda x: Get_transformation_smallest_deviation(x),axis=1) # applying the function
print(df4.shape)

df4['transformation_number'] = df4['transformation'].apply(lambda x: len(x.split('|')))
df4['groups_number'] = df4['groups'].apply(lambda x: len(x.split('|')))
print(df4.shape)
df4.head()

In [14]:
def compare(x):
    trans = str(x['transformation_number'])
    groups = str(x['groups_number'])
    return int(trans==groups)
df4['compare'] = df4.apply(lambda x: compare(x),axis=1)
print(df4.shape)
print(df4[df4['compare'] ==0].shape[0]) # this denotes that all the transformation_number same as groups. 
# # So no transformation or groups are removed from df3 after applying Get_transformation_smallest_deviation

In [15]:
# df3 has all possible transformation and cores for the chemical pairs forming MMPs.
MMPs = pd.DataFrame(df3[['Smi1','Smi2','iden1','iden2','transformation','core','iden_transform']])
print(MMPs.shape)
MMPs.head()

In [16]:
MMPs.to_csv('MMP_output.tsv',sep='\t',index=None) # output file